# 05Tools: Prediction - Online
Predictions from models created in the 05 series of notebooks.

This notebook is part of collection of examples that showcase many ways to serve models:
- Online:
    - (**THIS NOTEBOOK**) Vertex AI Endpoints: Python, REST, CLI (gcloud): [05Tools - Prediction - Online.ipynb](./05Tools%20-%20Prediction%20-%20Online.ipynb)
    - Local with TensorFlow ModelServer: [05Tools - Prediction - Local.ipynb](./05Tools%20-%20Prediction%20-%20Local.ipynb)
    - Custom: Build a custom container with TensorFlow ModelServer: [05Tools - Prediction - Custom.ipynb](./05Tools%20-%20Prediction%20-%20Custom.ipynb)
        - Remote Service with Cloud Run
        - Local Service with Docker Run
- Batch: [05Tools - Prediction - Batch.ipynb](./05Tools%20-%20Prediction%20-%20Batch.ipynb)
    - BigQuery ML Model Import
    - Vertex AI Batch Prediction Jobs

**Prerequisites:**
-  At least 1 of the notebooks in this series [05, 05a-05i]

**Conceptual Flow & Workflow**

<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05tools_pred_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05tools_pred_console.png" width="45%">
</p>

---
## Setup

inputs:

In [129]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [130]:
REGION = 'us-central1'
EXPERIMENT = '05_predictions'
SERIES = '05'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [131]:
from google.cloud import aiplatform
from google.cloud import bigquery

import tensorflow as tf

from datetime import datetime
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

import asyncio
import time
import multiprocessing

clients:

In [132]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

In [133]:
BUCKET = PROJECT_ID
DIR = f"temp/{EXPERIMENT}"

environment:

In [134]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Endpoint

[Endpoint Properties and Methods](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint):

```python
endpoint
endpoint.display_name
endpoint.resource_name
endpoint.traffic_split
endpoint.list_models()
```

In [135]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
endpoint = endpoints[0]

---
## Retrieve Records For Prediction

In [136]:
n = 1000
pred = bq.query(query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE splits='TEST' LIMIT {n}").to_dataframe()

In [137]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST


Remove columns not included as features in the model:

In [138]:
newobs = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')
#newobs[0]

In [139]:
len(newobs)

1000

---
## Online Predictions: Methods for Vertex AI Endpoints

There are multiple ways to interact with a Vertex AI Endpoint from Python.  This notebook gives example of for Python (multiple version of the client, and layers), as well as REST and the `gcloud` CLI.  To better understand these client review the notes here: [aiplatform_notes.md](../Tips/aiplatform_notes.md).

**Explanations**
For each of the methods below, the `predict` part of the request can be exchanged for `explain` if the endpoint has a model deployed with explanations setup.  Note: This will not work for the raw prediction methods.  See more about setting up explainability in the explainability notebooks within this series.
- [05Tools - Explainability - Example-Based.ipynb](./05Tools%20-%20Explainability%20-%20Example-Based.ipynb)
- [05Tools - Explainability - Feature-Based.ipynb](./05Tools%20-%20Explainability%20-%20Feature-Based.ipynb)

### Get Prediction: Python Client

In [140]:
instances = [json_format.ParseDict(newobs[0], Value())]

In [141]:
prediction = endpoint.predict(instances=instances)
prediction

Prediction(predictions=[[0.999359429, 0.000640570885]], deployed_model_id='6805735083375329280', model_version_id='1', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05_05h', explanations=None)

In [142]:
prediction.predictions[0]

[0.999359429, 0.000640570885]

In [143]:
np.argmax(prediction.predictions[0])

0

### Get Prediction: Python Client (gapic)

In [144]:
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

In [145]:
predictor = aiplatform.gapic.PredictionServiceClient(client_options = client_options)

In [146]:
instances = [json_format.ParseDict(newobs[0], Value())]

In [147]:
prediction = predictor.predict(
    endpoint = endpoint.resource_name,
    instances = instances
)
prediction

predictions {
  list_value {
    values {
      number_value: 0.999359429
    }
    values {
      number_value: 0.000640570885
    }
  }
}
deployed_model_id: "6805735083375329280"
model: "projects/1026793852137/locations/us-central1/models/model_05_05h"
model_display_name: "05_05h"
model_version_id: "1"

In [148]:
prediction.predictions[0]

[0.999359429, 0.000640570885]

In [149]:
np.argmax(prediction.predictions[0])

0

#### Use gapic for Raw Predictions

In [150]:
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

In [151]:
predictor = aiplatform.gapic.PredictionServiceClient(client_options = client_options)

In [152]:
from google.api import httpbody_pb2
instances = {"instances": [newobs[0]], "signature_name": "serving_default"}
http_body = httpbody_pb2.HttpBody(data = json.dumps(instances).encode("utf-8"), content_type = "application/json")

In [153]:
prediction = predictor.raw_predict(
    endpoint = endpoint.resource_name,
    http_body = http_body
)
prediction

content_type: "application/json"
data: "{\n    \"predictions\": [[0.999359429, 0.000640570885]\n    ]\n}"

In [154]:
prediction = json.loads(prediction.data)
prediction

{'predictions': [[0.999359429, 0.000640570885]]}

In [155]:
prediction['predictions'][0]

[0.999359429, 0.000640570885]

In [156]:
np.argmax(prediction['predictions'][0])

0

### Get Prediction: Python Client V1

In [157]:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

v1_client = aiplatform_v1.PredictionServiceClient(client_options = client_options)

In [158]:
instances = [json_format.ParseDict(newobs[0], Value())]

In [159]:
prediction = v1_client.predict(
    endpoint = endpoint.resource_name,
    instances = instances
)
prediction

predictions {
  list_value {
    values {
      number_value: 0.999359429
    }
    values {
      number_value: 0.000640570885
    }
  }
}
deployed_model_id: "6805735083375329280"
model: "projects/1026793852137/locations/us-central1/models/model_05_05h"
model_display_name: "05_05h"
model_version_id: "1"

In [160]:
prediction.predictions[0]

[0.999359429, 0.000640570885]

In [161]:
np.argmax(prediction.predictions[0])

0

#### Use Python Client V1 For Raw Predictions

In [162]:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

v1_client = aiplatform_v1.PredictionServiceClient(client_options = client_options)

In [163]:
from google.api import httpbody_pb2
instances = {"instances": [newobs[0]], "signature_name": "serving_default"}
http_body = httpbody_pb2.HttpBody(data = json.dumps(instances).encode("utf-8"), content_type = "application/json")

In [164]:
prediction = v1_client.raw_predict(
    endpoint = endpoint.resource_name,
    http_body = http_body
)
prediction

content_type: "application/json"
data: "{\n    \"predictions\": [[0.999359429, 0.000640570885]\n    ]\n}"

In [165]:
prediction = json.loads(prediction.data)
prediction

{'predictions': [[0.999359429, 0.000640570885]]}

In [166]:
prediction['predictions'][0]

[0.999359429, 0.000640570885]

In [167]:
np.argmax(prediction['predictions'][0])

0

### Get Prediction: Python Client V1 beta 1

In [168]:
from google.cloud import aiplatform_v1beta1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

v1beta1_client = aiplatform_v1beta1.PredictionServiceClient(client_options = client_options)

In [169]:
instances = [json_format.ParseDict(newobs[0], Value())]

In [170]:
prediction = v1beta1_client.predict(
    endpoint = endpoint.resource_name,
    instances = instances
)
prediction

predictions {
  list_value {
    values {
      number_value: 0.999359429
    }
    values {
      number_value: 0.000640570885
    }
  }
}
deployed_model_id: "6805735083375329280"
model: "projects/1026793852137/locations/us-central1/models/model_05_05h"
model_display_name: "05_05h"
model_version_id: "1"

In [171]:
prediction.predictions[0]

[0.999359429, 0.000640570885]

In [172]:
np.argmax(prediction.predictions[0])

0

#### Use Python Client V1 beta 1 For Raw Predictions

In [173]:
from google.cloud import aiplatform_v1beta1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}

v1beta1_client = aiplatform_v1.PredictionServiceClient(client_options = client_options)

In [174]:
from google.api import httpbody_pb2
instances = {"instances": [newobs[0]], "signature_name": "serving_default"}
http_body = httpbody_pb2.HttpBody(data = json.dumps(instances).encode("utf-8"), content_type = "application/json")

In [175]:
prediction = v1beta1_client.raw_predict(
    endpoint = endpoint.resource_name,
    http_body = http_body
)
prediction

content_type: "application/json"
data: "{\n    \"predictions\": [[0.999359429, 0.000640570885]\n    ]\n}"

In [176]:
prediction = json.loads(prediction.data)
prediction

{'predictions': [[0.999359429, 0.000640570885]]}

In [177]:
prediction['predictions'][0]

[0.999359429, 0.000640570885]

In [178]:
np.argmax(prediction['predictions'][0])

0

### Get Prediction: REST

#### Method 1: Command Line CURL

In [179]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newobs[0]]}))

In [180]:
prediction = !curl -s POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

prediction = json.loads(''.join([p.strip() for p in prediction]))
prediction

{'predictions': [[0.999359429, 0.000640570885]],
 'deployedModelId': '6805735083375329280',
 'model': 'projects/1026793852137/locations/us-central1/models/model_05_05h',
 'modelDisplayName': '05_05h',
 'modelVersionId': '1'}

In [181]:
prediction['predictions'][0]

[0.999359429, 0.000640570885]

In [182]:
np.argmax(prediction['predictions'][0])

0

##### Use CURL for Raw Predictions

In [183]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"signature_name": "serving_default", "instances": [newobs[0]]}))

In [184]:
prediction = !curl -s POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:rawPredict

prediction = json.loads(''.join([p.strip() for p in prediction]))
prediction

{'predictions': [[0.999359429, 0.000640570885]]}

In [185]:
prediction['predictions'][0]

[0.999359429, 0.000640570885]

In [186]:
np.argmax(prediction['predictions'][0])

0

#### Method 2: Python with requests

In [187]:
import requests

In [188]:
token = !gcloud auth application-default print-access-token
headers = {"content-type": "application/json; charset=utf-8", "Authorization": f'Bearer {token[0]}'}
json_response = requests.post(f'https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict', data=json.dumps({"instances": [newobs[0]]}), headers=headers)

In [189]:
print(json_response.text)

{
  "predictions": [
    [
      0.999359429,
      0.000640570885
    ]
  ],
  "deployedModelId": "6805735083375329280",
  "model": "projects/1026793852137/locations/us-central1/models/model_05_05h",
  "modelDisplayName": "05_05h",
  "modelVersionId": "1"
}



In [190]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999359429, 0.000640570885]]

In [191]:
np.argmax(predictions[0])

0

##### Use Requests for Raw Predictions

In [192]:
import requests

In [193]:
token = !gcloud auth application-default print-access-token
headers = {"content-type": "application/json; charset=utf-8", "Authorization": f'Bearer {token[0]}'}
json_response = requests.post(f'https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:rawPredict', data=json.dumps({"signature_name": "serving_default", "instances": [newobs[0]]}), headers=headers)

In [194]:
print(json_response.text)

{
    "predictions": [[0.999359429, 0.000640570885]
    ]
}


In [195]:
predictions = json.loads(json_response.text)['predictions']
predictions

[[0.999359429, 0.000640570885]]

In [196]:
np.argmax(predictions[0])

0

### Get Prediction: gcloud (CLI)

In [197]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newobs[0]]}))

In [198]:
prediction = !gcloud ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json
prediction

['Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]',
 '[[0.999359429, 0.000640570885]]']

In [199]:
import ast
prediction = ast.literal_eval(prediction[1])
prediction[0]

[0.999359429, 0.000640570885]

In [200]:
np.argmax(prediction[0])

0

#### Use gcloud (CLI) For Raw Predictions

In [201]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"signature_name": "serving_default", "instances": [newobs[0]]}))

In [202]:
prediction = !gcloud ai endpoints raw-predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --format="json" --request=@{DIR}/request.json
prediction

['Using endpoint [https://us-central1-aiplatform.googleapis.com/]',
 '{',
 '  "predictions": [',
 '    [',
 '      0.999359429,',
 '      0.000640570885',
 '    ]',
 '  ]',
 '}']

In [203]:
prediction = json.loads("".join(prediction[1:]))
prediction

{'predictions': [[0.999359429, 0.000640570885]]}

In [204]:
prediction['predictions'][0]

[0.999359429, 0.000640570885]

In [205]:
np.argmax(prediction['predictions'][0])

0

---
## Requesting Many Predictions: Synchronous and Asynchronus

There are times where you want to make many request of an endpoint for predictions.  If you send request one at a time, synchronous, then the endpoint will fullfill each request as it receives it.  An endpoint is designed to handle simoultaneous requests, asynchronous.  If you set the max_replicas > 1 during the endpoint setup then it will also scale up to handle the amount of traffic.  

Using Python to make concurrent request is an example of multiprocessing.  A review of the tip notebook [Python Multiprocessing](../Tips/Python%20Multiprocessing.ipynb) can be helpful for understanding the method used below to make asynchronous requests concurrently using `asyncio`.

---
### Online Predictions: Synchronous Examples
Synchronous calls to the Vertex AI Endpoint with different batch size of instances.  This is packaging multiple prediction request up in a single call of the endpoint - size is batch_size.

In [102]:
def syncPredictions(batch_size = 1):
    predictions = []
    start = time.perf_counter()
    # a loop where each step request predictions for batch_size number of instances - in a single request
    for p in range(0, len(newobs), batch_size):
        instances = [json_format.ParseDict(example, Value()) for example in newobs[p:p+batch_size]]
        preds = endpoint.predict(instances = instances)
        predictions.extend(np.argmax(pred) for pred in preds.predictions)
    elapsed = time.perf_counter() - start
    print(f'{elapsed:0.5f} seconds')
    return predictions

In [103]:
# default batch_size = 1
predictions = syncPredictions()

14.42168 seconds


In [104]:
# specify batch_size = 1 - identical to default here
predictions = syncPredictions(1)

16.45706 seconds


In [105]:
# specify batch_size = 1 - identical to default here
predictions = syncPredictions(batch_size = 1)

19.25611 seconds


In [106]:
# specify batch_size = 2 - expecting half the time if the endpoint can handle multiple at the same time
predictions = syncPredictions(batch_size = 2)

10.31313 seconds


In [107]:
# specify batch_size = 10 - expecting 1/10 the time if the endpoint can handle this many at the same time
predictions = syncPredictions(batch_size = 10)

2.25430 seconds


In [108]:
# get a count of the number of predictions that resulted in 0 (not fraud) and 1 (fraud)
from collections import Counter
c = Counter(predictions)
c

Counter({0: 998, 1: 2})

In [109]:
# get the index for the predictions that resulted in predicting = 1 (fraud)
[i for i, j in enumerate(predictions) if j == 1]

[53, 576]

In [110]:
# review the inputs that lead to a prediction = 1 (fraud)
pred.iloc[[i for i, j in enumerate(predictions) if j == 1]]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
53,85285,-7.030308,3.421991,-9.525072,5.270891,-4.024630,-2.865682,-6.989195,3.791551,-4.622730,...,0.036943,-0.355519,0.353634,1.042458,1.359516,-0.272188,0.0,1,0a3b566f-e662-4cd0-b702-99299890cb0f,TEST
576,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,e17f6ee4-8dd8-4a38-9f51-e1bbf1e1aa2a,TEST


---
### Online Predictions: Asynchronous Examples
Use asyncio for concurrent requests of different batch sizes.  Like the synchronous example above, this makes request with batch_size number of prediction instances.  This approach also manages multiple request concurrently to further increase parallelism.

In [111]:
# get the Async Client for the endpoint:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
parent = f"projects/{PROJECT_ID}/locations/{REGION}"

client = aiplatform_v1.PredictionServiceAsyncClient(client_options = client_options)

In [112]:
endpoint.resource_name

'projects/1026793852137/locations/us-central1/endpoints/1961322035766362112'

#### Test a single instance

In [115]:
instance = [json_format.ParseDict(newobs[0], Value())]

In [116]:
await client.predict(endpoint = endpoint.resource_name, instances = instances)

predictions {
  list_value {
    values {
      number_value: 0.999359429
    }
    values {
      number_value: 0.000640570885
    }
  }
}
deployed_model_id: "6805735083375329280"
model: "projects/1026793852137/locations/us-central1/models/model_05_05h"
model_display_name: "05_05h"
model_version_id: "1"

#### Try all the instances in combinations of batch_size and concur_requests

In [117]:
async def asyncPredictions(batch_size = 1, concur_requests = 100):
    limit = asyncio.Semaphore(concur_requests)

    predictions = [None] * len(newobs)

    async def predictor(p, newob):
        instances = [json_format.ParseDict(example, Value()) for example in newob]
        async with limit:
            prediction = await client.predict(endpoint = endpoint.resource_name, instances = instances)
            if limit.locked():
                await asyncio.sleep(.01)

        predictions[p:p+batch_size] = [np.argmax(pred) for pred in prediction.predictions]

    async def runner(newobs):
        tasks = []
        for p in range(0, len(newobs), batch_size):
            task = asyncio.create_task(predictor(p, newobs[p:p+batch_size]))
            tasks.append(task)

        results = await asyncio.gather(*tasks)

    start = time.perf_counter()
    await runner(newobs)
    elapsed = time.perf_counter() - start
    print(f'{elapsed:0.5f} seconds')
    
    return predictions

In [119]:
# try all the instances in separate request (batch_size = 1)
# may need to run a second time - the initial ramp can throw an error
predictions = await asyncPredictions()

0.97373 seconds


In [120]:
# manage 100 concurrent request max where each has a single instance (batch_size = 1)
predictions = await asyncPredictions(1, 100)

0.97975 seconds


In [121]:
# manage 100 concurrent request max where each has a single instance (batch_size = 1)
predictions = await asyncPredictions(batch_size = 1, concur_requests = 100)

0.97210 seconds


In [122]:
# manage 100 concurrent request max where each has a 2 instances (batch_size = 2)
predictions = await asyncPredictions(batch_size = 2, concur_requests = 100)

0.63126 seconds


In [123]:
# manage 100 concurrent request max where each has a 10 instances (batch_size = 10)
predictions = await asyncPredictions(batch_size = 10, concur_requests = 100)

0.28745 seconds


In [124]:
len(predictions)

1000

In [126]:
# get a count of the number of predictions that resulted in 0 (not fraud) and 1 (fraud)
from collections import Counter
c = Counter(predictions)
c

Counter({0: 998, 1: 2})

In [127]:
# get the index for the predictions that resulted in predicting = 1 (fraud)
[i for i, j in enumerate(predictions) if j == 1]

[53, 576]

In [128]:
# review the inputs that lead to a prediction = 1 (fraud)
pred.iloc[[i for i, j in enumerate(predictions) if j == 1]]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
53,85285,-7.030308,3.421991,-9.525072,5.270891,-4.024630,-2.865682,-6.989195,3.791551,-4.622730,...,0.036943,-0.355519,0.353634,1.042458,1.359516,-0.272188,0.0,1,0a3b566f-e662-4cd0-b702-99299890cb0f,TEST
576,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,e17f6ee4-8dd8-4a38-9f51-e1bbf1e1aa2a,TEST
